# IMPORT, CONFIG

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/data_nir')

In [3]:
!pip install transformers
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.6 MB/s 
     |████████████████████████████████| 7.6 MB 64.5 MB/s 
     |████████████████████████████████| 163 kB 66.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import BertForSequenceClassification, AutoTokenizer, logging

import os
import random
import time
from tqdm import tqdm
import pickle
import gensim

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from torchinfo import summary

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report
from collections import defaultdict
from textwrap import wrap

import warnings
warnings.filterwarnings("ignore")
logging.set_verbosity_error()

In [5]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 8, 6

In [6]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
torch.cuda.random.manual_seed_all(RANDOM_SEED)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
DEVICE

device(type='cuda')

In [8]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'

In [9]:
!nvidia-smi

Mon Nov 14 19:04:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    12W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# LOAD DATA

In [10]:
target_dir = "./output_bert_Ohsumed"

In [11]:
with open('ohsumed_dataset.pkl','rb') as ohsumed_file:
     source = pickle.load(ohsumed_file)

In [12]:
source

,texts,Y
0,Haemophilus influenzae meningitis with prolong...,0
1,Augmentation mentoplasty using Mersilene mesh....,0
2,Multiple intracranial mucoceles associated wit...,0
3,Replacement of an aortic valve cusp after neon...,0
4,Mucosal intussusception to avoid ascending cho...,0
...,...,...
56979,Ionized calcium in blood: studies on patients ...,22
56980,Effects of immediate postoperative enteral nut...,22
56981,Effects of enteral fat emulsion on fat absorpt...,22
56982,Inhibition of early atherogenesis in transgeni...,22


In [13]:
source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56984 entries, 0 to 56983
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texts   56984 non-null  object
 1   Y       56984 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 890.5+ KB


In [14]:
target_column = 'Y'

In [15]:
train_df, non_train_df = train_test_split(source, test_size=0.2, random_state=RANDOM_SEED,stratify=source[target_column].to_list())

In [16]:
dev_df, test_df = train_test_split(non_train_df, test_size=0.5, random_state=RANDOM_SEED,stratify=non_train_df[target_column].to_list())

In [17]:
#train_df, non_train_df = train_test_split(source, test_size=0.8, random_state=RANDOM_SEED,stratify=source[target_column].to_list())

In [18]:
#dev_df, non_dev_df = train_test_split(non_train_df, test_size=0.95, random_state=RANDOM_SEED,stratify=non_train_df[target_column].to_list())

In [19]:
#test_df, non_test_df = train_test_split(non_dev_df, test_size=0.95, random_state=RANDOM_SEED,stratify=non_dev_df[target_column].to_list())

In [20]:
len(train_df), len(dev_df), len(test_df)

(45587, 5698, 5699)

In [21]:
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# MODEL

In [22]:
class BertModel(nn.Module):
    def __init__(self, requires_grad = True):
        super(BertModel, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME,num_labels = len(source.groupby(target_column).count().index))
        self.tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=True)
        self.requires_grad = requires_grad
        self.device = torch.device(DEVICE)
        for param in self.bert.parameters():
            param.requires_grad = requires_grad  # Each parameter requires gradient

    def forward(self, batch_seqs, batch_seq_masks, batch_seq_segments, labels):
        loss, logits = self.bert(input_ids = batch_seqs, attention_mask = batch_seq_masks, 
                              token_type_ids=batch_seq_segments, labels = labels)[:2]
        probabilities = nn.functional.softmax(logits, dim=-1)
        return loss, logits, probabilities

In [23]:
bertmodel = BertModel(requires_grad = True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [24]:
summary(bertmodel)

Layer (type:depth-idx)                                       Param #
BertModel                                                    --
├─BertForSequenceClassification: 1-1                         --
│    └─BertModel: 2-1                                        --
│    │    └─BertEmbeddings: 3-1                              23,837,184
│    │    └─BertEncoder: 3-2                                 85,054,464
│    │    └─BertPooler: 3-3                                  590,592
│    └─Dropout: 2-2                                          --
│    └─Linear: 2-3                                           17,687
Total params: 109,499,927
Trainable params: 109,499,927
Non-trainable params: 0

In [25]:
bertmodel.bert.base_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [26]:
bertmodel.bert.classifier

Linear(in_features=768, out_features=23, bias=True)

In [27]:
model = bertmodel.to(DEVICE)

# PREPROCESSING

In [28]:
tokenizer = bertmodel.tokenizer

In [29]:
tokenizer.sep_token, tokenizer.sep_token_id

('[SEP]', 102)

In [30]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 101)

In [31]:
tokenizer.pad_token, tokenizer.pad_token_id

('[PAD]', 0)

# DATASET

In [32]:
class DataPrecessForSentence(Dataset):
    """
    Encoding sentences
    """
    def __init__(self, bert_tokenizer, df, max_seq_len = 50):
        super(DataPrecessForSentence, self).__init__()
        self.bert_tokenizer = bert_tokenizer
        self.max_seq_len = max_seq_len
        self.input_ids, self.attention_mask, self.token_type_ids, self.labels = self.get_input(df)
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.token_type_ids[idx], self.labels[idx]
        
    # Convert dataframe to tensor
    def get_input(self, df):
        sentences = df['texts'].values
        labels = df[target_column].values
        
        # tokenizer
        tokens_seq = list(map(self.bert_tokenizer.tokenize, sentences)) # list of shape [sentence_len, token_len]
        
        # Get fixed-length sequence and its mask
        result = list(map(self.trunate_and_pad, tokens_seq))
        
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]
        
        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(labels).type(torch.long)
               )
    
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.bert_tokenizer.convert_tokens_to_ids(tokens_seq)
        input_ids += padding   
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len)
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids

In [33]:
# sentences = (train_df['texts'].values, dev_df['texts'].values, test_df['texts'].values)
# sp = (list(map(tokenizer.tokenize, sentences[0])), list(map(tokenizer.tokenize, sentences[1])), list(map(tokenizer.tokenize, sentences[2])))
# len_sp = (list(map(len,sp[0])),list(map(len,sp[1])),list(map(len,sp[2])))
# max_sp = max(len_sp[0]), max(len_sp[1]), max(len_sp[2])
# if max(max_sp) > 512:
#     MAX_LEN = 512
# else:
#     MAX_LEM = max(max_sp)
# del sentences, sp, len_sp, max_sp
MAX_LEN = 512
MAX_LEN

512

In [34]:
BATCH_SIZE = 12

In [35]:
train_data = DataPrecessForSentence(tokenizer, train_df, max_seq_len = MAX_LEN)
train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)

In [36]:
dev_data = DataPrecessForSentence(tokenizer,dev_df, max_seq_len = MAX_LEN)
dev_loader = DataLoader(dev_data, shuffle=True, batch_size=BATCH_SIZE)

In [37]:
test_data = DataPrecessForSentence(tokenizer,test_df, max_seq_len = MAX_LEN)
test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)

# TRAIN

In [38]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
LR = 2e-05
optimizer_grouped_parameters = [
    {
    'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay':0.01
    },
    {
    'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    'weight_decay':0.0
    }
]
optimizer = optim.AdamW(optimizer_grouped_parameters, lr=LR)

In [39]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.85, patience=0)

In [50]:
def Metric(y_true, y_pred):
    """
    compute and show the classification result
    """
    accuracy = accuracy_score(y_true, y_pred)
    macro_precision = precision_score(y_true, y_pred, average='macro')
    macro_recall = recall_score(y_true, y_pred, average='macro')
    weighted_f1 = f1_score(y_true, y_pred, average='macro')
    target_names = [f'class_{i}' for i in set(source[target_column].values)]
    report = classification_report(y_true, y_pred, target_names=target_names, digits=3)

    print('Accuracy: {:.1%}\nPrecision: {:.1%}\nRecall: {:.1%}\nF1: {:.1%}'.format(accuracy, macro_precision,
                                           macro_recall, weighted_f1))
    print("classification_report:\n")
    print(report)
  
  
def correct_predictions(output_probabilities, targets):
    """
    Compute the number of predictions that match some target classes in the
    output of a model.
    Args:
        output_probabilities: A tensor of probabilities for different output
            classes.
        targets: The indices of the actual target classes.
    Returns:
        The number of correct predictions in 'output_probabilities'.
    """
    _, out_classes = output_probabilities.max(dim=1)
    correct = (out_classes == targets).sum()
    return correct.item()


def train(model, dataloader, optimizer, epoch_number, max_gradient_norm):
    """
    Train a model for one epoch on some input data with a given optimizer and
    criterion.
    Args:
        model: A torch module that must be trained on some input data.
        dataloader: A DataLoader object to iterate over the training data.
        optimizer: A torch optimizer to use for training on the input model.
        epoch_number: The number of the epoch for which training is performed.
        max_gradient_norm: Max. norm for gradient norm clipping.
    Returns:
        epoch_time: The total time necessary to train the epoch.
        epoch_loss: The training loss computed for the epoch.
        epoch_accuracy: The accuracy computed for the epoch.
    """
    # Switch the model to train mode.
    model.train()
    device = model.device
    epoch_start = time.time()
    batch_time_avg = 0.0
    running_loss = 0.0
    correct_preds = 0
    tqdm_batch_iterator = tqdm(dataloader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate(tqdm_batch_iterator):
        batch_start = time.time()
        seqs, masks, segments, labels = batch_seqs.to(device), batch_seq_masks.to(device), batch_seq_segments.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        loss, logits, probabilities = model(seqs, masks, segments, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step()
        batch_time_avg += time.time() - batch_start
        running_loss += loss.item()
        correct_preds += correct_predictions(probabilities, labels)
        description = "Avg. batch proc. time: {:.4f}s, loss: {:.4f}"\
                      .format(batch_time_avg/(batch_index+1), running_loss/(batch_index+1))
        tqdm_batch_iterator.set_description(description)
    epoch_time = time.time() - epoch_start
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct_preds / len(dataloader.dataset)
    return epoch_time, epoch_loss, epoch_accuracy


def validate(model, dataloader):
    """
    Compute the loss and accuracy of a model on some validation dataset.
    Args:
        model: A torch module for which the loss and accuracy must be
            computed.
        dataloader: A DataLoader object to iterate over the validation data.
    Returns:
        epoch_time: The total time to compute the loss and accuracy on the
            entire validation set.
        epoch_loss: The loss computed on the entire validation set.
        epoch_accuracy: The accuracy computed on the entire validation set.
        roc_auc_score(all_labels, all_prob): The auc computed on the entire validation set.
        all_prob: The probability of classification as label 1 on the entire validation set.
    """
    # Switch to evaluate mode.
    model.eval()
    device = model.device
    epoch_start = time.time()
    running_loss = 0.0
    running_accuracy = 0.0
    all_prob = []
    all_labels = []
    # Deactivate autograd for evaluation.
    with torch.no_grad():
        for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in dataloader:
            # Move input and output data to the GPU if one is used.
            seqs = batch_seqs.to(device)
            masks = batch_seq_masks.to(device)
            segments = batch_seq_segments.to(device)
            labels = batch_labels.to(device)
            loss, logits, probabilities = model(seqs, masks, segments, labels)
            running_loss += loss.item()
            running_accuracy += correct_predictions(probabilities, labels)
            all_prob.extend(probabilities.cpu().numpy())
            all_labels.extend(batch_labels)
    epoch_time = time.time() - epoch_start
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = running_accuracy / (len(dataloader.dataset))
    return epoch_time, epoch_loss, epoch_accuracy, roc_auc_score(all_labels, all_prob,multi_class='ovr'), all_prob
    #return epoch_time, epoch_loss, epoch_accuracy, all_prob

def test(model, dataloader):
    """
    Test the accuracy of a model on some labelled test dataset.
    Args:
        model: The torch module on which testing must be performed.
        dataloader: A DataLoader object to iterate over some dataset.
    Returns:
        batch_time: The average time to predict the classes of a batch.
        total_time: The total time to process the whole dataset.
        accuracy: The accuracy of the model on the input data.
        all_prob: The probability of classification as label 1 on the entire validation set.
    """
    # Switch the model to eval mode.
    model.eval()
    device = model.device
    time_start = time.time()
    batch_time = 0.0
    accuracy = 0.0
    all_prob = []
    all_labels = []
    # Deactivate autograd for evaluation.
    with torch.no_grad():
        for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in dataloader:
            batch_start = time.time()
            # Move input and output data to the GPU if one is used.
            seqs, masks, segments, labels = batch_seqs.to(device), batch_seq_masks.to(device), batch_seq_segments.to(device), batch_labels.to(device)
            _, _, probabilities = model(seqs, masks, segments, labels)
            accuracy += correct_predictions(probabilities, labels)
            batch_time += time.time() - batch_start
            all_prob.extend(probabilities.cpu().numpy())
            all_labels.extend(batch_labels)
    batch_time /= len(dataloader)
    total_time = time.time() - time_start
    accuracy /= (len(dataloader.dataset))

    return batch_time, total_time, accuracy, all_prob

In [41]:
best_score = 0.0
epochs= 3
start_epoch = 1
patience = 1
max_grad_norm = 10.0
if_save_model = True
checkpoint = os.path.join(target_dir, "bert_base_uncased_ohsumed_epoch2.tar") # None

# Data for loss curves plot
epochs_count = []
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []
valid_aucs = []

if checkpoint:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint["epoch"] + 1
    best_score = checkpoint["best_score"]
    print("\t* Training will continue on existing model from epoch {}...".format(start_epoch))
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    epochs_count = checkpoint["epochs_count"]
    train_losses = checkpoint["train_losses"]
    train_accuracy = checkpoint["train_accuracy"]
    valid_losses = checkpoint["valid_losses"]
    valid_accuracy = checkpoint["valid_accuracy"]
    valid_auc = checkpoint["valid_auc"]

 # Compute loss and accuracy before starting (or resuming) training.
_, valid_loss, valid_accuracy, auc, _ = validate(model, dev_loader)
print("\n* Validation loss before training: {:.4f}, accuracy: {:.4f}%, auc: {:.4f}".format(valid_loss, (valid_accuracy*100), auc))

# -------------------- Training epochs -----------------------------------#

print("\n", 20 * "=", "Training bert model on device: {}".format(DEVICE), 20 * "=")
patience_counter = 0
for epoch in range(start_epoch, epochs + 1):
    epochs_count.append(epoch)

    print("* Training epoch {}:".format(epoch))
    epoch_time, epoch_loss, epoch_accuracy = train(model, train_loader, optimizer, epoch, max_grad_norm)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)  
    print("-> Training time: {:.4f}s, loss = {:.4f}, accuracy: {:.4f}%".format(epoch_time, epoch_loss, (epoch_accuracy*100)))

    print("* Validation for epoch {}:".format(epoch))
    epoch_time, epoch_loss, epoch_accuracy , epoch_auc, _, = validate(model, dev_loader)
    valid_losses.append(epoch_loss)
    valid_accuracies.append(epoch_accuracy)
    valid_aucs.append(epoch_auc)
    print("-> Valid. time: {:.4f}s, loss: {:.4f}, accuracy: {:.4f}%, auc: {:.4f}\n"
          .format(epoch_time, epoch_loss, (epoch_accuracy*100), epoch_auc))

    # Update the optimizer's learning rate with the scheduler.
    scheduler.step(epoch_accuracy)
    ## scheduler.step()

    # Early stopping on validation accuracy.
    if epoch_accuracy < best_score:
        patience_counter += 1
    else:
        best_score = epoch_accuracy
        patience_counter = 0
        if (if_save_model):
            torch.save({"epoch": epoch, 
                       "model": model.state_dict(),
                       "optimizer": optimizer.state_dict(),
                       "best_score": best_score,
                       "epochs_count": epochs_count,
                       "train_losses": train_losses,
                       "train_accuracy": train_accuracies,
                       "valid_losses": valid_losses,
                       "valid_accuracy": valid_accuracies,
                       "valid_auc": valid_aucs
                       },
                       os.path.join(target_dir, "best.pth.tar"))
            print("save model succesfully!\n")

        # run model on test set and save the prediction result to csv
        print("* Test for epoch {}:".format(epoch))
        _, _, test_accuracy, _, all_prob = validate(model, test_loader)
        print("Test accuracy: {:.4f}%\n".format(test_accuracy))
        columns_names = [f'prob_{i}' for i in range(all_prob[0].shape[0])]
        test_prediction = pd.DataFrame(all_prob,columns=columns_names)
        test_prediction['prediction'] = test_prediction.apply(lambda x: columns_names.index(x.idxmax()) , axis=1)
        test_prediction = test_prediction[[*columns_names, 'prediction']]
        test_prediction.to_csv(os.path.join(target_dir,"test_prediction.csv"), index=False,sep=';')

    if patience_counter >= patience:
        print("-> Early stopping: patience limit reached, stopping...")
        break

	* Training will continue on existing model from epoch 3...

* Validation loss before training: 1.5031, accuracy: 46.6479%, auc: 0.9284

 ==================== Training bert model on device: cuda ====================
* Training epoch 3:


Avg. batch proc. time: 1.1399s, loss: 1.3568: 100%|██████████| 3799/3799 [1:12:41<00:00,  1.15s/it]


-> Training time: 4361.4499s, loss = 1.3568, accuracy: 51.0738%
* Validation for epoch 3:
-> Valid. time: 199.2621s, loss: 1.5186, accuracy: 45.3317%, auc: 0.9324

-> Early stopping: patience limit reached, stopping...


In [42]:
test_result = pd.read_csv(os.path.join(target_dir, 'test_prediction.csv'),sep=';')

In [51]:
Metric(test_df.Y, test_result.prediction) 

Accuracy: 45.4%
Precision: 42.8%
Recall: 42.6%
F1: 40.8%
classification_report:

              precision    recall  f1-score   support

     class_0      0.394     0.307     0.345       254
     class_1      0.303     0.085     0.133       117
     class_2      0.765     0.310     0.441        42
     class_3      0.500     0.695     0.582       633
     class_4      0.415     0.393     0.404       168
     class_5      0.421     0.676     0.519       299
     class_6      0.474     0.346     0.400        52
     class_7      0.414     0.456     0.434       259
     class_8      0.301     0.704     0.422        71
     class_9      0.415     0.548     0.472       385
    class_10      0.471     0.560     0.511       100
    class_11      0.490     0.508     0.499       252
    class_12      0.350     0.546     0.427       163
    class_13      0.645     0.564     0.602       610
    class_14      0.358     0.305     0.329       128
    class_15      0.458     0.349     0.396       109


# PREDICTION

In [54]:
def model_load_test(test_df, target_dir, test_prediction_dir, test_prediction_name, max_seq_len=50, batch_size=32):
    """
    Parameters
    ----------
    test_df : pandas dataframe of test set.
    target_dir : the path of pretrained model.
    test_prediction_dir : the path that you want to save the prediction result to.
    test_prediction_name : the file name of the prediction result.
    max_seq_len: the max truncated length.
    batch_size : the default is 32.
    
    """
    
    bertmodel = BertModel(requires_grad = False)
    tokenizer = bertmodel.tokenizer
    device = torch.device("cuda")
    
    print(20 * "=", " Preparing for testing ", 20 * "=")
    if platform == "linux" or platform == "linux2":
        checkpoint = torch.load(os.path.join(target_dir, "bert_base_uncased_ohsumed_epoch2.tar"))
    else:
        checkpoint = torch.load(os.path.join(target_dir, "bert_base_uncased_ohsumed_epoch2.tar"), map_location=device)
        
    print("\t* Loading test data...")    
    test_data = DataPrecessForSentence(tokenizer,test_df, max_seq_len = max_seq_len) 
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

    # Retrieving model parameters from checkpoint.
    print("\t* Building model...")
    model = bertmodel.to(device)
    model.load_state_dict(checkpoint["model"])
    print(20 * "=", " Testing BERT model on device: {} ".format(device), 20 * "=")
    
    batch_time, total_time, accuracy, all_prob = test(model, test_loader)
    print("\n-> Average batch processing time: {:.4f}s, total test time: {:.4f}s, accuracy: {:.4f}%\n".format(batch_time, total_time, (accuracy*100)))
    
    columns_names = [f'prob_{i}' for i in range(all_prob[0].shape[0])]
    test_prediction = pd.DataFrame(all_prob,columns=columns_names)
    test_prediction['prediction'] = test_prediction.apply(lambda x: columns_names.index(x.idxmax()) , axis=1)
    if not os.path.exists(test_prediction_dir):
        os.makedirs(test_prediction_dir)
    test_prediction.to_csv(os.path.join(test_prediction_dir, test_prediction_name), index=False)

In [55]:
platform = 'Windows'
model_load_test(test_df, target_dir, target_dir, "test_prediction2.csv", max_seq_len=MAX_LEN, batch_size=BATCH_SIZE)

====================  Preparing for testing  ====================
	* Loading test data...
	* Building model...
====================  Testing BERT model on device: cuda  ====================

-> Average batch processing time: 0.3949s, total test time: 187.8394s, accuracy: 46.0607%

